In [1]:
from surprise import Dataset
from surprise.prediction_algorithms import knns
from surprise import KNNBasic
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
import pandas as pd
import numpy as np
from surprise.model_selection import train_test_split

In [9]:
# Load the movielens-100k dataset
data = Dataset.load_builtin('ml-100k')

In [12]:
# Retrieve the trainset.
trainset = data.build_full_trainset()

In [13]:
# Build an algorithm, and train it.
algo = KNNBasic()
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [14]:
uid = str(196)  # raw user id (as in the ratings file). They are **strings**!
iid = str(302)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

user: 196        item: 302        r_ui = 4.00   est = 4.06   {'actual_k': 40, 'was_impossible': False}


The predict() uses raw ids (please read this about raw and inner ids). As the dataset we have used has been read from a file, the raw ids are strings (even if they represent numbers).

In [15]:
trainset, testset = train_test_split(data, test_size=0.2)

In [16]:
print('Type trainset :',type(trainset),'\n')
print('Type testset :',type(testset))

Type trainset : <class 'surprise.trainset.Trainset'> 

Type testset : <class 'list'>


Notice how there is no X_train or y_train in our values here. Our only features here are the ratings of other users and items, so we need to keep everything together. What is happening in the train-test split here is that surprise is randomly selecting certain $r_{ij}$ for users $u_{i}$ and items $i_{j} $. 80% of the ratings are in the training set and 20% in the test set. Let's investigate trainset and testset further.
Interestingly enough, the values here are of different data types! The trainset is still a surprise specific data type that is optimized for computational efficiency and the testset is a standard Python list - you'll see why when we start making predictions. Let's take a look at how large our testset is as well as what's contained in an individual element. A sacrifice of surprise's implementation is that we lose a lot of the exploratory methods that are present with Pandas.

In [17]:
print(len(testset))
print(testset[0])

20000
('898', '288', 4.0)


## Item-based KNN

In [18]:
print('Number of users: ', trainset.n_users, '\n')
print('Number of items: ', trainset.n_items, '\n')

Number of users:  943 

Number of items:  1653 



In [19]:
sim_pearson = {'name':'pearson', 'user_based':False}
basic_pearson = knns.KNNBasic(sim_options=sim_pearson)
basic_pearson.fit(trainset)
predictions = basic_pearson.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.0399
1.0399397956794532


The next model we're going to try is KNN with Means. This is the same thing as the basic KNN model, except it takes into account the mean rating of each user or item depending on whether you are performing user-user or item-item similarities, respectively.

In [23]:
basic_pearson2 = knns.KNNBasic(k=50, sim_options=sim_pearson)
basic_pearson2.fit(trainset)
predictions2 = basic_pearson2.test(testset)
print(accuracy.rmse(predictions2))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.0349
1.0348823988355307


In [16]:
sim_pearson = {'name':'pearson', 'user_based':False}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson)
knn_means.fit(trainset)
predictions = knn_means.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9442
0.9442301838392885


## User-based KNN

In [33]:
sim_cos = {'name':'cosine', 'user_based':True}
basic_cos = knns.KNNBasic(sim_options=sim_cos)
basic_cos.fit(trainset)
predictions2 = basic_cos.test(testset)
print(accuracy.rmse(predictions2))

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0228
1.0227970369365185


## Tuning the Algorithm Parameters

Surprise provides a GridSearchCV class analogous to GridSearchCV from scikit-learn.

With a dict of all parameters, GridSearchCV tries all the combinations of parameters and reports the best parameters for any accuracy measure

‘min_support’: The minimum number of public items (when 'user_based' is 'True') or the minimum number of public users (when 'user_based' is 'False'), the similarity is not zero. 


In [35]:
from surprise.model_selection import GridSearchCV

sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(knns.KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi